<a href="https://colab.research.google.com/github/cviddenKwantum/KwantumDev/blob/main/Brand_density.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brand density
- Chad Vidden, COO [Kwantum Analytics](https://www.kwantumanalytics.com/)
- June 2023

## Resources:
- https://github.com/openai/openai-cookbook
- https://github.com/eon01/awesome-chatgpt
- https://github.com/openai/openai-cookbook/blob/main/examples/azure/chat.ipynb
- https://github.com/taishi-i/awesome-ChatGPT-repositories

In [1]:
#@title
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.3 MB/s eta 0:00:00


In [2]:
!pip install tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.8 MB/s eta 0:00:00


In [16]:
#@title
import openai
import pandas as pd
from io import StringIO
import pickle
from IPython.core.display import HTML
import re
import time
import os
from openai.embeddings_utils import get_embedding
import tiktoken
from sklearn.manifold import TSNE
import numpy as np
import random
from google.colab import drive
import json
from datetime import datetime

drive.mount('/content/drive')

with open('/content/drive/MyDrive/GPT_SECRET_KEY.json') as f:
    data = json.load(f)
openai.api_key = data["API_KEY"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# helper functions
def get_completion_chat(messages, temp, model="gpt-3.5-turbo"):
    while True:
        try:
            response = openai.ChatCompletion.create(
              model=model,
              messages=messages,
              temperature=temp, # this is the degree of randomness of the model's output
              )
            return response.choices[0].message["content"]
        except openai.error.RateLimitError as error:
              print('Server overloaded. Retrying in 5 seconds...')
              time.sleep(5)
              get_completion(prompt)

def get_completion(prompt, max_length = 3500, model="text-davinci-003"):
    while True:
        try:
            response = openai.Completion.create(
                model=model,
                prompt=prompt,
                temperature=0.7,
                max_tokens=max_length,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0)
            return response.get('choices',[])[0]['text']
        except openai.error.RateLimitError as error:
              print('Server overloaded. Retrying in 5 seconds...')
              time.sleep(5)
              get_completion(prompt)

def makelist(input_string):
  pattern = r'- (.+)'
  matches = re.findall(pattern, input_string)
  bullet_list = list(matches)
  return bullet_list

### Step 1:  Category description

Run a loop a few times to pick up a variety of words or phrases to describe the category to get as exhaustive list ~ 70-100

In [9]:
prompt1 = f"""What words, attributes, companies, or phrases come to mind when \
you think of the talent acquisition or hiring category? Please list everything \
that comes to mind. Let's make sure your to understand answer is as exhaustive \
as possible."""

In [10]:
messages = [{"role": "system", "content": "you are an AI assistant who helps a \
human to understand what the associations are with a category."},
            {"role": "user", "content": prompt1}]
res = get_completion_chat(messages, 0.15)
messages.append({"role": "assistant", "content": res})
print(res)

Sure, here are some words, attributes, companies, and phrases that come to mind when I think of the talent acquisition or hiring category:

- Recruitment
- Job search
- Candidate sourcing
- Applicant tracking systems (ATS)
- Job boards
- Staffing agencies
- Human resources (HR)
- Hiring managers
- Interviews
- Resumes/CVs
- Job descriptions
- Employer branding
- Diversity and inclusion
- Talent management
- Employee retention
- Onboarding
- LinkedIn
- Glassdoor
- Indeed
- Monster
- ZipRecruiter
- CareerBuilder
- Hired
- Workable
- Greenhouse
- Lever
- Jobvite
- ATS integrations
- Recruitment marketing
- Employee referrals
- Background checks
- Pre-employment assessments
- Job fairs
- Networking events
- Salary negotiations
- Remote hiring
- Freelance hiring
- Gig economy
- Employer of choice
- Talent pipeline
- Recruitment metrics
- Talent acquisition strategy
- Employer-employee relationship
- Job satisfaction
- Employee engagement
- Succession planning
- Workforce planning
- Talent a

In [11]:
res1list = makelist(res)
print(res1list)
print(len(res1list))

['Recruitment', 'Job search', 'Candidate sourcing', 'Applicant tracking systems (ATS)', 'Job boards', 'Staffing agencies', 'Human resources (HR)', 'Hiring managers', 'Interviews', 'Resumes/CVs', 'Job descriptions', 'Employer branding', 'Diversity and inclusion', 'Talent management', 'Employee retention', 'Onboarding', 'LinkedIn', 'Glassdoor', 'Indeed', 'Monster', 'ZipRecruiter', 'CareerBuilder', 'Hired', 'Workable', 'Greenhouse', 'Lever', 'Jobvite', 'ATS integrations', 'Recruitment marketing', 'Employee referrals', 'Background checks', 'Pre-employment assessments', 'Job fairs', 'Networking events', 'Salary negotiations', 'Remote hiring', 'Freelance hiring', 'Gig economy', 'Employer of choice', 'Talent pipeline', 'Recruitment metrics', 'Talent acquisition strategy', 'Employer-employee relationship', 'Job satisfaction', 'Employee engagement', 'Succession planning', 'Workforce planning', 'Talent acquisition technology', 'Recruitment automation', 'Recruitment chatbots', 'Recruitment analyt

In [12]:
prompt2 = f"""What are some words, attributes, companies, or phrases missing \
from the list? Let's try and be as exhaustive as possible, but focus on what \
is top of mind."""

In [13]:
messages.append({"role": "user", "content": prompt2})
res = get_completion_chat(messages, 0.15)
messages.append({"role": "assistant", "content": res})
print(res)

Here are some additional words, attributes, companies, or phrases that come to mind:

- Job market
- Talent shortage
- Skills gap
- Employer value proposition (EVP)
- Candidate experience
- Employer reviews
- Social media recruiting
- Employee referrals
- Diversity recruiting
- Inclusive hiring
- Campus recruiting
- Internship programs
- Employee benefits
- Compensation packages
- Job offer letters
- Background screening
- Drug testing
- Reference checks
- Job analysis
- Job evaluation
- Job design
- Job posting
- Job fair
- Recruitment process outsourcing (RPO)
- Recruitment advertising
- Recruitment events
- Recruitment budget
- Recruitment funnel
- Recruitment metrics
- Recruitment ROI
- Recruitment software
- Recruitment training
- Recruitment trends
- Recruitment challenges
- Recruitment best practices
- Recruitment strategies
- Recruitment branding
- Recruitment communication
- Recruitment conversion rate
- Recruitment conversion funnel
- Recruitment conversion optimization
- Rec

In [14]:
res2list = makelist(res)
print(res2list)
print(len(res2list))

['Job market', 'Talent shortage', 'Skills gap', 'Employer value proposition (EVP)', 'Candidate experience', 'Employer reviews', 'Social media recruiting', 'Employee referrals', 'Diversity recruiting', 'Inclusive hiring', 'Campus recruiting', 'Internship programs', 'Employee benefits', 'Compensation packages', 'Job offer letters', 'Background screening', 'Drug testing', 'Reference checks', 'Job analysis', 'Job evaluation', 'Job design', 'Job posting', 'Job fair', 'Recruitment process outsourcing (RPO)', 'Recruitment advertising', 'Recruitment events', 'Recruitment budget', 'Recruitment funnel', 'Recruitment metrics', 'Recruitment ROI', 'Recruitment software', 'Recruitment training', 'Recruitment trends', 'Recruitment challenges', 'Recruitment best practices', 'Recruitment strategies', 'Recruitment branding', 'Recruitment communication', 'Recruitment conversion rate', 'Recruitment conversion funnel', 'Recruitment conversion optimization', 'Recruitment conversion tracking', 'Recruitment c

In [15]:
joined_list = list(set(res1list + res2list)) # keep unique words/phrases
print(joined_list)
print(len(joined_list))

['LinkedIn', 'Job descriptions', 'Recruitment conversion funnel', 'Recruitment branding', 'Recruitment conversion improvement', 'Networking events', 'Recruitment conversion rate optimization', 'Talent management', 'Recruitment chatbots', 'Greenhouse', 'Job market', 'Job fair', 'Job analysis', 'Recruitment metrics', 'Employer of choice', 'Job fairs', 'Hiring managers', 'Recruitment training', 'Recruitment process outsourcing (RPO)', 'Reference checks', 'Inclusive hiring', 'Job boards', 'Recruitment conversion rate optimization.', 'Recruitment best practices', 'Employer-employee relationship', 'Recruitment automation', 'Internship programs', 'Succession planning', 'Applicant tracking systems (ATS)', 'Diversity and inclusion', 'Talent acquisition strategy', 'Background screening', 'Recruitment events', 'Recruitment conversion rate analysis', 'Job offer letters', 'Recruitment trends', 'Resumes/CVs', 'Employee engagement', 'Campus recruiting', 'Recruitment conversion testing', 'Recruitment 

### Step 2: Refine list

Remove overlap from Step 1 list.

In [16]:
prompt4 = f"""Review the list of words and phrases that people associate with the  \
talent acquisition or hiring category and remove the one that are similar \
so we reduce overlap.

LIST OF WORDS: {joined_list}"""

In [17]:
messages = [{"role": "system", "content": "you are an AI assistant who helps a \
human to understand what the associations are with a category."},
            {"role": "user", "content": prompt4}]
res = get_completion_chat(messages, .15)
messages.append({"role": "assistant", "content": res})
print(res)

Here is the list of words and phrases with similar meanings that can be removed to reduce overlap:

- Job fair and Job fairs
- Recruitment conversion rate optimization and Recruitment conversion rate improvement
- Recruitment conversion rate testing and Recruitment conversion testing
- Recruitment conversion rate tracking and Recruitment conversion tracking
- Recruitment conversion analysis and Recruitment conversion rate analysis

The revised list of words and phrases associated with the talent acquisition or hiring category is:

- LinkedIn
- Job descriptions
- Recruitment conversion funnel
- Recruitment branding
- Networking events
- Talent management
- Recruitment chatbots
- Greenhouse
- Job market
- Recruitment metrics
- Employer of choice
- Hiring managers
- Recruitment training
- Recruitment process outsourcing (RPO)
- Reference checks
- Inclusive hiring
- Job boards
- Recruitment best practices
- Employer-employee relationship
- Recruitment automation
- Internship programs
- Suc

In [18]:
res3list = makelist(res)
print(res3list)
print(len(res3list))

['Job fair and Job fairs', 'Recruitment conversion rate optimization and Recruitment conversion rate improvement', 'Recruitment conversion rate testing and Recruitment conversion testing', 'Recruitment conversion rate tracking and Recruitment conversion tracking', 'Recruitment conversion analysis and Recruitment conversion rate analysis', 'LinkedIn', 'Job descriptions', 'Recruitment conversion funnel', 'Recruitment branding', 'Networking events', 'Talent management', 'Recruitment chatbots', 'Greenhouse', 'Job market', 'Recruitment metrics', 'Employer of choice', 'Hiring managers', 'Recruitment training', 'Recruitment process outsourcing (RPO)', 'Reference checks', 'Inclusive hiring', 'Job boards', 'Recruitment best practices', 'Employer-employee relationship', 'Recruitment automation', 'Internship programs', 'Succession planning', 'Applicant tracking systems (ATS)', 'Diversity and inclusion', 'Talent acquisition strategy', 'Background screening', 'Recruitment events', 'Job offer letter

In [19]:
with open("/content/drive/MyDrive/Colab Notebooks/catList.pkl", "wb") as file:
  pickle.dump(res3list, file)

In [5]:
with open("/content/drive/MyDrive/Colab Notebooks/catList.pkl", "rb") as file:
    catList = pickle.load(file)

print(catList)
print(len(catList))

['Job fair and Job fairs', 'Recruitment conversion rate optimization and Recruitment conversion rate improvement', 'Recruitment conversion rate testing and Recruitment conversion testing', 'Recruitment conversion rate tracking and Recruitment conversion tracking', 'Recruitment conversion analysis and Recruitment conversion rate analysis', 'LinkedIn', 'Job descriptions', 'Recruitment conversion funnel', 'Recruitment branding', 'Networking events', 'Talent management', 'Recruitment chatbots', 'Greenhouse', 'Job market', 'Recruitment metrics', 'Employer of choice', 'Hiring managers', 'Recruitment training', 'Recruitment process outsourcing (RPO)', 'Reference checks', 'Inclusive hiring', 'Job boards', 'Recruitment best practices', 'Employer-employee relationship', 'Recruitment automation', 'Internship programs', 'Succession planning', 'Applicant tracking systems (ATS)', 'Diversity and inclusion', 'Talent acquisition strategy', 'Background screening', 'Recruitment events', 'Job offer letter

## Step 3: Rank associations by frequency

In [44]:
prompt5 = f""" Review the list of words and phrases below that describe the \
talent acquisition or hiring category and rank them from highest to lowest by\
frequency of customer mention. Please estimate the proportion of mentions and \
add in parentheses. Let's try and provide the best estimate possible. Give \ 
exact proprtions of customer mentions for each category.

LIST OF WORDS AND PHRASES: {catList}

OUTPUT FORMAT: 
1. Phrase1 (mention proportion %)
2. Prase2 (mention proportion %)
etc
"""

In [45]:
for i in range(2):

  print(i)

  messages = [{"role": "system", "content": "you are an AI assistant who helps a \
  human to understand what the associations are with a category."},
              {"role": "user", "content": prompt5}]
  res = get_completion_chat(messages, .15)
  messages.append({"role": "assistant", "content": res})

  # parse result as dataframe
  pattern = r'\d+\.\s+(.+)'
  matches = re.findall(pattern, res, re.MULTILINE)
  parsed_list = [item.strip() for item in matches]

  pattern = r'(.+)\s+\((\d+%)\)'
  parsed_data = [re.match(pattern, item) for item in parsed_list]

  categories = [match.group(1) for match in parsed_data]
  mention_proportions = [match.group(2) for match in parsed_data]

  df = pd.DataFrame({'category': categories, 'mention proportion str': mention_proportions})
  df['mention proportion'] = df['mention proportion str'].str.replace('%', '').astype(float) / 100
  df = df.drop('mention proportion str', axis=1)

  # save output
  fileName = "mentionProp"+datetime.now().strftime("%Y-%m-%d %H:%M:%S")+".pkl"

  with open("/content/drive/MyDrive/Colab Notebooks/MentionProps/"+fileName, "wb") as file:
    pickle.dump(df, file)

0
1


In [46]:
print(res)

Based on the list of words and phrases provided, here is an estimate of the frequency of customer mention for each category, ranked from highest to lowest:

1. Job boards (10%)
2. Applicant tracking systems (ATS) (8%)
3. Recruitment process outsourcing (RPO) (7%)
4. Candidate experience (6%)
5. Employer branding (5%)
6. Social media recruiting (4%)
7. Employee referrals (4%)
8. Recruitment analytics (3%)
9. Job fairs (3%)
10. Diversity and inclusion (3%)
11. Talent acquisition strategy (3%)
12. Onboarding (2%)
13. Recruitment budget (2%)
14. Recruitment software (2%)
15. Employee retention (2%)
16. Background checks (2%)
17. Employer value proposition (EVP) (2%)
18. Remote hiring (2%)
19. Recruitment advertising (2%)
20. Interviews (2%)
21. Human resources (HR) (2%)
22. Recruitment conversion rate optimization (2%)
23. Recruitment conversion funnel (2%)
24. Recruitment conversion tracking (2%)
25. Recruitment conversion rate testing (2%)
26. Recruitment conversion rate tracking (2%)
27

In [47]:
display(df)

,category,mention proportion
0,Job boards,0.10
1,Applicant tracking systems (ATS),0.08
2,Recruitment process outsourcing (RPO),0.07
3,Candidate experience,0.06
4,Employer branding,0.05
...,...,...
84,Recruitment communication,0.02
85,Indeed,0.02
86,Talent acquisition technology,0.02
87,Recruitment marketing,0.02


## Step 4: Comentions of phrases

In [51]:
prompt6 = f""" Review the list of words and phrases below and determine which \
of these get frequently mentioned together in a sentence or paragraph. \
Please list the words and phrases that get mentioned together with other words \
and phrases from the same list. Please estimate the proportion of phrase \
comentions and add in parentheses. Let's' try and provide the best estimate \
possible.

LIST OF WORDS AND PHRASES: {catList}

OUTPUT FORMAT: 
1. Phrase1 mentioned with Phrase2 (comention proportion %)
1. Phrase3 mentioned with Phrase4 (comention proportion %)
etc
"""

In [52]:
messages = [{"role": "system", "content": "you are an AI assistant who helps a \
  human to understand what the associations are with a category."},
              {"role": "user", "content": prompt6}]
res = get_completion_chat(messages, .15)
messages.append({"role": "assistant", "content": res})
print(res)

1. Recruitment conversion rate optimization and Recruitment conversion rate improvement mentioned together (100%)
2. Recruitment conversion rate testing and Recruitment conversion testing mentioned together (100%)
3. Recruitment conversion rate tracking and Recruitment conversion tracking mentioned together (100%)
4. Recruitment conversion analysis and Recruitment conversion rate analysis mentioned together (100%)
5. Job descriptions mentioned with Recruitment branding (50%)
6. Recruitment conversion funnel mentioned with Recruitment metrics (50%)
7. Talent management mentioned with Succession planning (50%)
8. Recruitment chatbots mentioned with Recruitment automation (50%)
9. Job market mentioned with Recruitment trends (50%)
10. Employer of choice mentioned with Employee engagement (50%)
11. Recruitment process outsourcing (RPO) mentioned with Staffing agencies (50%)
12. Inclusive hiring mentioned with Diversity and inclusion (100%)
13. Job boards mentioned with Social media recruit